In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import sqlite3

In [12]:
df_lecture = pd.read_csv('general_courses.csv', encoding='utf_8_sig')
df_lecture['teacher_course'] = df_lecture['teacher'] + ' - ' + df_lecture['course_name']
df_lecture = df_lecture.reindex(columns=['id', 'teacher', 'course_name', 'teacher_course', 'assignment', 'test', 'in_group', 'attend', 'report'])
df_lecture = df_lecture.drop(labels=range(270, 350), axis=0)
df_lecture['ratings'] = ''
df_lecture

,id,teacher,course_name,teacher_course,assignment,test,in_group,attend,report,ratings
0,1,林偉淑,文學名篇選讀：愛戀與生活,林偉淑 - 文學名篇選讀：愛戀與生活,4.0,6.0,10,0.0,0.0,
1,2,趙衛民,文學名篇選讀：愛戀與生活,趙衛民 - 文學名篇選讀：愛戀與生活,0.0,10.0,0,1.0,0.0,
2,3,黃麗卿,文學名篇選讀：愛戀與生活,黃麗卿 - 文學名篇選讀：愛戀與生活,0.0,6.0,10,2.0,2.0,
3,4,陳大道,文學名篇選讀：愛戀與生活(全英語授課),陳大道 - 文學名篇選讀：愛戀與生活(全英語授課),0.0,8.0,10,2.0,0.0,
4,5,洪婕寧,文學名篇選讀：愛戀與生活(遠距非同步課程),洪婕寧 - 文學名篇選讀：愛戀與生活(遠距非同步課程),1.5,3.0,10,3.0,3.0,
...,...,...,...,...,...,...,...,...,...,...
265,266,陳旭,科學之旅(雙語授課(中文/英文)),陳旭 - 科學之旅(雙語授課(中文/英文)),7.5,0.0,0,2.5,0.0,
266,267,吳俊毅,科學之旅(全英語授課),吳俊毅 - 科學之旅(全英語授課),5.5,3.0,0,1.5,0.0,
267,268,姜杰,統計與生活,姜杰 - 統計與生活,7.0,0.0,0,3.0,0.0,
268,269,王芸馨,生命科學：基因科技與健康,王芸馨 - 生命科學：基因科技與健康,3.0,6.0,0,1.0,0.0,


In [13]:
types = ['assignment', 'test', 'in_group', 'attend', 'report']
s = []
for i in range(len(df_lecture)):
    for type in types:
        s.append(df_lecture.at[i, type])
    df_lecture.at[i, 'ratings'] = s
    s = []
df_lecture

,id,teacher,course_name,teacher_course,assignment,test,in_group,attend,report,ratings
0,1,林偉淑,文學名篇選讀：愛戀與生活,林偉淑 - 文學名篇選讀：愛戀與生活,4.0,6.0,10,0.0,0.0,"[4.0, 6.0, 10, 0.0, 0.0]"
1,2,趙衛民,文學名篇選讀：愛戀與生活,趙衛民 - 文學名篇選讀：愛戀與生活,0.0,10.0,0,1.0,0.0,"[0.0, 10.0, 0, 1.0, 0.0]"
2,3,黃麗卿,文學名篇選讀：愛戀與生活,黃麗卿 - 文學名篇選讀：愛戀與生活,0.0,6.0,10,2.0,2.0,"[0.0, 6.0, 10, 2.0, 2.0]"
3,4,陳大道,文學名篇選讀：愛戀與生活(全英語授課),陳大道 - 文學名篇選讀：愛戀與生活(全英語授課),0.0,8.0,10,2.0,0.0,"[0.0, 8.0, 10, 2.0, 0.0]"
4,5,洪婕寧,文學名篇選讀：愛戀與生活(遠距非同步課程),洪婕寧 - 文學名篇選讀：愛戀與生活(遠距非同步課程),1.5,3.0,10,3.0,3.0,"[1.5, 3.0, 10, 3.0, 3.0]"
...,...,...,...,...,...,...,...,...,...,...
265,266,陳旭,科學之旅(雙語授課(中文/英文)),陳旭 - 科學之旅(雙語授課(中文/英文)),7.5,0.0,0,2.5,0.0,"[7.5, 0.0, 0, 2.5, 0.0]"
266,267,吳俊毅,科學之旅(全英語授課),吳俊毅 - 科學之旅(全英語授課),5.5,3.0,0,1.5,0.0,"[5.5, 3.0, 0, 1.5, 0.0]"
267,268,姜杰,統計與生活,姜杰 - 統計與生活,7.0,0.0,0,3.0,0.0,"[7.0, 0.0, 0, 3.0, 0.0]"
268,269,王芸馨,生命科學：基因科技與健康,王芸馨 - 生命科學：基因科技與健康,3.0,6.0,0,1.0,0.0,"[3.0, 6.0, 0, 1.0, 0.0]"


In [4]:
# con = sqlite3.connect('general_courses.db')
# cur = con.cursor()
# with open('temp_GC.csv', 'r', encoding='utf_8_sig') as file:
#     for row in file:
#         cur.execute('INSERT INTO General_courses VALUES (?,?,?,?,?,?,?,?,?,?)', row.split(','))
#         con.commit()
# con.close()

In [12]:
# for i in range(len(df_lecture.index)):
#     if int(df_lecture.iloc[i, 4]) == 1:
#         df_lecture.iloc[i, 4] = float(10)

In [14]:
types = ['assignment', 'report', 'test', 'in_group', 'attend']
s = []
features = {}
for i in range(1, len(df_lecture.index)):
    for type in types:
        s.append(df_lecture.at[i, type])
    features[df_lecture.at[i, 'teacher_course']] = s
    s = []
features

{'趙衛民 - 文學名篇選讀：愛戀與生活': [0.0, 0.0, 10.0, 0, 1.0],
 '黃麗卿 - 文學名篇選讀：愛戀與生活': [0.0, 2.0, 6.0, 10, 2.0],
 '陳大道 - 文學名篇選讀：愛戀與生活(全英語授課)': [0.0, 0.0, 8.0, 10, 2.0],
 '洪婕寧 - 文學名篇選讀：愛戀與生活(遠距非同步課程)': [1.5, 3.0, 3.0, 10, 3.0],
 '謝旻琪 - 文學名篇選讀  : 愛戀與生活': [3.0, 0.0, 6.0, 0, 1.0],
 '高柏園 - 文學名篇選讀：群己與生命': [0.0, 4.0, 4.0, 10, 2.0],
 '陳淑玲 - 文學名篇選讀：群己與生命': [6.0, 0.0, 0.0, 0, 4.0],
 '黃文倩 - 文學名篇選讀：群己與生命': [0.0, 4.0, 4.0, 10, 2.0],
 '孟慶延 - 文學名篇選讀：群己與生命': [1.0, 0.0, 7.0, 0, 2.0],
 '侯如綺 - 台灣文學選讀': [4.0, 3.0, 3.0, 10, 0.0],
 '林淑瑩 - 科幻小說選讀': [2.0, 0.0, 7.0, 0, 1.0],
 '陳宜武 - 當代英美文學經典': [2.5, 0.0, 5.0, 0, 2.5],
 '吳凱書 - 當代英美文學經典(全英語授課)': [1.5, 1.0, 6.0, 10, 1.5],
 '顧錦芬 - 日本文學中譯賞析': [4.0, 3.0, 3.0, 0, 3.0],
 '邱若山 - 日本文學中譯賞析': [0.0, 3.3, 3.3, 0, 3.4],
 '林盛彬 - 西班牙語文學經典賞析': [3.0, 3.0, 3.0, 10, 1.0],
 '林郁嫺 - 德語文學名著選讀（一）': [0.0, 7.0, 0.0, 10, 3.0],
 '龔信賢 - 俄國文學經典入門': [5.0, 0.0, 5.0, 0, 0.0],
 '廖潤珮 - 法國文學、生活與文化': [5.0, 3.0, 1.0, 10, 2.0],
 '李建程 - 法國經典文學導讀': [3.0, 1.0, 6.0, 10, 3.0],
 '葉夌 - 村上春樹講座': [9.0, 0.0, 0.0, 0, 1.0],
 '

In [15]:
# def recommend(course_name):
#     temp = {}
#     for course in features.keys():
#         temp[course] = cosine_similarity([features[course_name]], [features[course]])[0][0]
#     temp.pop(course_name)
#     top = sorted(temp.items(), key=lambda x: x[1], reverse=True)
#     top = top[1:12]
#     for i in range(10):
#         print(top[i][0])
#     print(top)
# recommend('姜杰 - 統計與生活')

[('王伯昌 - 科技未來', 0.9999999999999998), ('陳旭 - 科學之旅(雙語授課(中文/英文))', 0.9965457582448795), ('徐佐銘 - 醫學倫理', 0.987241120712647), ('徐佐銘 - 環境倫理', 0.987241120712647), ('徐佐銘 - 性愛的文學與哲學', 0.987241120712647), ('徐佐銘 - 環境倫理(進)', 0.987241120712647), ('沈禎 - 書法藝術的應用', 0.987241120712647), ('史城均 - 社會未來(全英語授課)', 0.987241120712647), ('張朝欽 - 科技未來', 0.987241120712647), ('賴盛維 - 網路與資訊科技', 0.987241120712647), ('周文祺 - 生活中的化學', 0.987241120712647)]


In [23]:
def recommend(course_name):
    temp = pd.DataFrame(columns=['course', 'similarity'], index=range(len(features)))
    for i, course in enumerate(features.keys(), start=1):
        sim = cosine_similarity([features[course_name]], [features[course]])[0][0]
        temp.loc[i] = [course, sim]
    temp = temp.sort_values(by='similarity', ascending=False)
    temp = temp.reset_index(drop=True)
    temp = temp.drop(temp[temp['course'] == course_name].index)
    return temp.loc[:10, ['course', 'similarity']]
Recommendations = recommend('姜杰 - 統計與生活')

In [24]:
Recommendations

,course,similarity
1,王伯昌 - 科技未來,1.0
2,劉亞蘭 - 性別理論與實踐,1.0
3,陳旭 - 科學之旅(雙語授課(中文/英文)),0.996546
4,賴盛維 - 網路與資訊科技,0.987241
5,沈禎 - 書法藝術的應用,0.987241
6,史城均 - 社會未來(全英語授課),0.987241
7,張朝欽 - 科技未來,0.987241
8,徐佐銘 - 環境倫理,0.987241
9,徐佐銘 - 性愛的文學與哲學,0.987241
10,徐佐銘 - 環境倫理(進),0.987241
